In [1]:
!pip install ../..

Defaulting to user installation because normal site-packages is not writeable
Processing /opt/jupyter/notebook/dev/dudyrev/FCApy
  Created wheel for fcapy: filename=fcapy-0.1.0-py3-none-any.whl size=68513 sha256=70677a6cd60fc65741018d4a9baaa6da270d7f5137d56507770f606dfa8af8e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-690yx7oq/wheels/54/75/56/6d37d7ed7625138e16d04981b1bdbbab9942388f67d015cdf0
Successfully built fcapy
  Attempting uninstall: fcapy
    Found existing installation: fcapy 0.1.0
    Uninstalling fcapy-0.1.0:
      Successfully uninstalled fcapy-0.1.0


In [2]:
import pandas as pd

# Load the datasets

In [3]:
data_dict = {}

## California housing

In [4]:
from sklearn.datasets import fetch_california_housing

In [5]:
data = fetch_california_housing(as_frame=True)

ds = data['data']
y_feat = data['target_names'][0]
ds[y_feat] = data['target']

data_dict['calhouse'] = {
    'ds': ds,
    'train_feats': data['feature_names'],
    'cat_feats': [],
    'y_feat': y_feat
}

### Boston

In [6]:
from sklearn.datasets import load_boston

In [7]:
data = load_boston()

y_feat = 'price'
fs = data['feature_names']
ds = pd.DataFrame(data['data'], columns=fs)
ds[y_feat] =data['target']

data_dict['boston'] = {
    'ds': ds,
    'train_feats': fs,
    'cat_feats': [],
    'y_feat': y_feat
}

### Diabetes

In [8]:
from sklearn.datasets import load_diabetes

In [9]:
data = load_diabetes(as_frame=True)

ds = data['data']
y_feat = 'disease'
ds[y_feat] = data['target']

data_dict['diabetes'] = {
    'ds': ds,
    'train_feats': data['feature_names'],
    'cat_feats': [],
    'y_feat': y_feat
}

In [10]:
from sklearn.preprocessing import LabelEncoder

for data_name in data_dict.keys():
    ds = data_dict[data_name]['ds']
    cat_feats = data_dict[data_name]['cat_feats']
    ds[cat_feats] = ds[cat_feats].astype(str)
    for f in cat_feats:
        ds[f+'_le'] = LabelEncoder().fit_transform(ds[f])
    data_dict[data_name]['ds'] = ds
    #train_feats_le = [f+('_le' if f in cat_feats else '') for f in train_feats]

# Test models

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import Pool, CatBoostRegressor

from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from datetime import datetime

from tqdm import tqdm

In [12]:
import numpy as np
from fcapy.mvcontext.mvcontext import MVContext
from fcapy.mvcontext import pattern_structure as PS
from fcapy.ml.decision_lattice import DecisionLatticeRegressor

from collections.abc import Iterable

In [13]:
def test_catboost(X_train, X_test, y_train, y_test, cat_feats, params):
    cb = CatBoostRegressor(
        iterations=params.get('n_estimators'),
        max_depth=params.get('max_depth'),
        thread_count=params.get('n_jobs')
    )
    
    t1 = datetime.now()
    pool_train = Pool(X_train, y_train, cat_features=cat_feats)
    pool_test = Pool(X_test, cat_features=cat_feats)
    
    cb.fit(pool_train, verbose=False, plot=False)
    t2 = datetime.now()
    dt = (t2-t1).total_seconds()
    
    preds_train = cb.predict(pool_train)
    preds_test = cb.predict(pool_test)
    
    return dt, preds_train, preds_test, cb

In [14]:
def predict_by_fca(dl, X):
    pattern_types = {f: PS.IntervalPS for f in X.columns}
    mvctx = MVContext(
        X.values,
        attribute_names=X.columns,
        pattern_types=pattern_types
    )
    preds = dl.predict(mvctx)
    preds = [dl.lattice.top_concept.measures['mean_y'] if x is None else x for x in preds]
    
    return preds

def test_fca(X_train, X_test, y_train, y_test, params):
    dl = DecisionLatticeRegressor(**params)
    
    t1 = datetime.now()
    pattern_types = {f: PS.IntervalPS for f in X_train.columns}
    mvctx_train = MVContext(
        X_train.values, target=y_train.values,
        attribute_names=X_train.columns,
        pattern_types=pattern_types
    )
    dl.fit(mvctx_train)
    t2 = datetime.now()
    dt = (t2-t1).total_seconds()
    
    preds_train = predict_by_fca(dl, X_train)
    preds_test = predict_by_fca(dl, X_test)
    
    return dt, preds_train, preds_test, dl

In [15]:
import inspect

In [16]:
def test_sklearn_like(X_train, X_test, y_train, y_test, cls, params):
    cls_params = inspect.signature(cls.__init__).parameters
    cls_params = {k: v for k,v in params.items() if k in cls_params}
    
    model = cls(**cls_params)
    
    t1 = datetime.now()
    model.fit(X_train, y_train)
    t2 = datetime.now()
    dt = (t2-t1).total_seconds()
    
    preds_train = model.predict(X_train)
    preds_test = model.predict(X_test)
    
    return dt, preds_train, preds_test, model

In [17]:
def test_model(X_train, X_test, y_train, y_test, cls, cat_feats, params):
    if cls == CatBoostRegressor:
        output = test_catboost(X_train, X_test, y_train, y_test, cat_feats, params)
    elif cls == DecisionLatticeRegressor:
        output = test_fca(X_train, X_test, y_train, y_test, params)
    else:
        output = test_sklearn_like(X_train, X_test, y_train, y_test, cls, params)
    return output

In [18]:
def calc_metrics(y_train, y_test, preds_train, preds_test):
    metrics_dict = {}
    for m_name, m_func in [
        ('mse', mean_squared_error),
        ('mae', mean_absolute_error),
        ('r2', r2_score)
    ]:
        for ds_type in ['train', 'test']:
            y = y_train if ds_type == 'train' else y_test
            p = preds_train if ds_type == 'train' else preds_test
            metrics_dict[f"{m_name}_{ds_type}"] = m_func(y, p)

    return metrics_dict

In [19]:
def create_stat_dict(cls, data_name, kf_idx, train_idxs, test_idxs, params, metrics_dict, dt):
    stat = {
        'model': cls.__name__,
        'ds': data_name,
        'fold_id': kf_idx,
        'train_size': len(train_idxs),
        'test_size': len(test_idxs),
        'time': dt
    }
    stat = dict(stat, **{f"param_{k}":v for k,v in params.items()})
    stat = dict(stat, **{f"metric_{k}":v for k,v in metrics_dict.items()})
    return stat

In [20]:
random_state = 42

model_params_vars = [
    (DecisionTreeRegressor, dict()),
    (RandomForestRegressor, dict(n_estimators=5, max_depth=10)),
    (RandomForestRegressor, dict()),
    (GradientBoostingRegressor, dict()),
    (LGBMRegressor, dict()),
    (CatBoostRegressor, dict()),
    (DecisionLatticeRegressor,
     dict(
        algo='RandomForest',
        algo_params={'rf_params':{'n_estimators':5, 'max_depth':10,}}
     )),
    (DecisionLatticeRegressor,
     dict(
        algo='RandomForest',
        algo_params={'rf_params':{'n_estimators':10, 'max_depth':10,}}
     )),
    (DecisionLatticeRegressor,
     dict(
         algo='Sofia',
         algo_params={'L_max': 100}
     )
    )
]
model_params_vars = [
    (cls, dict({'random_state':random_state}, **params))
    for cls, params in model_params_vars
]

In [ ]:
%%time

kf = KFold(n_splits=5)
stat_ds = []

for data_name, data in tqdm(data_dict.items(), desc='iterate datasets'):
    ds, train_feats, cat_feats, y_feat = data['ds'], data['train_feats'], data['cat_feats'], data['y_feat']
    train_feats_le = [f+('_le' if f in cat_feats else '') for f in train_feats]

    for kf_idx, idxs in tqdm(enumerate(kf.split(ds[train_feats])), desc='KFold', total=kf.n_splits, leave=False):
        train_idxs, test_idxs = idxs
        ds_train, ds_test = ds.loc[train_idxs], ds.loc[test_idxs]
        
        #if kf_idx==1:
        #    break
        
        for cls, params in tqdm(model_params_vars, leave=False, desc='iterate models'):
            if cls == DecisionLatticeRegressor and params['algo'] == 'Sofia' and len(train_idxs)>5000:
                continue
            
            train_feats_ = train_feats if cls == CatBoostRegressor else train_feats_le
            
            res = test_model(
                ds_train[train_feats_], ds_test[train_feats_], ds_train[y_feat], ds_test[y_feat],
                cls, cat_feats, params
            )
            dt, preds_train, preds_test, model = res
            
            metrics_dict = calc_metrics(ds_train[y_feat], ds_test[y_feat], preds_train, preds_test)
            
            stat = create_stat_dict(cls, data_name, kf_idx, train_idxs, test_idxs, params, metrics_dict, dt)
            stat_ds.append(pd.Series(stat))
            
            if cls == DecisionLatticeRegressor:
                model.use_generators = True
                pattern_types = {f: PS.IntervalPS for f in train_feats_}
                mvctx = MVContext(
                    ds_train[train_feats_].values, pattern_types, 
                    attribute_names=train_feats_,
                )
                
                for gen_algo in ['approximate', 'exact']:
                    if gen_algo == 'exact' and params['algo'] == 'RandomForest' \
                        and params['algo_params']['rf_params']['n_estimators'] > 10:
                        continue
                        
                    if gen_algo == 'exact' and len(train_idxs) > 3000:
                        continue
                        
                    params = dict(params, gen_algo=gen_algo)
                    
                    t1 = datetime.now()
                    model.compute_generators(mvctx, gen_algo, use_tqdm=gen_algo=='exact')
                    t2 = datetime.now()
                    dt_gen = (t2-t1).total_seconds()
                    
                    preds_train = predict_by_fca(model, ds_train[train_feats_])
                    preds_test = predict_by_fca(model, ds_test[train_feats_])
                    metrics_dict = calc_metrics(ds_train[y_feat], ds_test[y_feat], preds_train, preds_test)
                    stat = create_stat_dict(cls, data_name, kf_idx, train_idxs, test_idxs, params, metrics_dict, dt+dt_gen)
                    stat_ds.append(pd.Series(stat))
                        
                    
        pd.concat(stat_ds,1, sort=False).T.to_csv('tmp_evaluation_regr.csv')
pd.concat(stat_ds,1, sort=False).T.to_csv('evaluation_regr_full1.csv')
!rm tmp_evaluation_regr.csv

# Compute Weighted Average Percentage Error post factum

In [21]:
stat_ds = pd.read_csv('evaluation_regr_full1.csv', index_col=0)
stat_ds.head()

,model,ds,fold_id,train_size,test_size,time,param_random_state,metric_mse_train,metric_mse_test,metric_mae_train,metric_mae_test,metric_r2_train,metric_r2_test,param_n_estimators,param_max_depth,param_algo,param_algo_params,param_gen_algo,metric_wape_train,metric_wape_test
0,DecisionTreeRegressor,calhouse,0,16512,4128,0.151587,42,1.051528e-31,0.793655,4.867984e-17,0.623827,1.000000,0.261216,NaN,NaN,NaN,NaN,NaN,2.248564e-17,0.370648
1,RandomForestRegressor,calhouse,0,16512,4128,0.277177,42,2.019904e-01,0.599036,3.110937e-01,0.580684,0.850314,0.442380,5.0,10.0,NaN,NaN,NaN,1.436969e-01,0.345015
2,RandomForestRegressor,calhouse,0,16512,4128,9.033681,42,3.465075e-02,0.521694,1.203863e-01,0.543827,0.974322,0.514374,NaN,NaN,NaN,NaN,NaN,5.560749e-02,0.323116
3,GradientBoostingRegressor,calhouse,0,16512,4128,2.807599,42,2.649648e-01,0.427120,3.613307e-01,0.523329,0.803647,0.602410,NaN,NaN,NaN,NaN,NaN,1.669018e-01,0.310937
4,LGBMRegressor,calhouse,0,16512,4128,200.734206,42,1.627876e-01,0.370611,2.791399e-01,0.446897,0.879366,0.655012,NaN,NaN,NaN,NaN,NaN,1.289371e-01,0.265525


In [22]:
stat_ds[f'metric_wape_train'] = None
stat_ds[f'metric_wape_test'] = None

In [23]:
kf = KFold(n_splits=5)

for data_name, data in tqdm(data_dict.items(), desc='iterate datasets'):
    ds, train_feats, cat_feats, y_feat = data['ds'], data['train_feats'], data['cat_feats'], data['y_feat']

    for kf_idx, idxs in tqdm(enumerate(kf.split(ds[train_feats])), desc='KFold', total=kf.n_splits, leave=False):
        train_idxs, test_idxs = idxs
        y_train = ds.loc[train_idxs, y_feat]
        y_test = ds.loc[test_idxs, y_feat]
        
        flg = (stat_ds['ds']==data_name)&(stat_ds['fold_id']==kf_idx)
        stat_ds.loc[flg, 'metric_wape_train'] = stat_ds.loc[flg, 'metric_mae_train']/y_train.mean()
        stat_ds.loc[flg, 'metric_wape_test'] = stat_ds.loc[flg, 'metric_mae_test']/y_test.mean()

KFold:   0%|          | 0/5 [00:00<?, ?it/s]
                                            
KFold:   0%|          | 0/5 [00:00<?, ?it/s]
                                            
iterate datasets: 100%|██████████| 3/3 [00:00<00:00, 25.78it/s]


In [24]:
stat_ds.to_csv('evaluation_regr_full1.csv')